# Construção da base de notícias ESG a partir da captura do Google RSS

- Autor: Daniel Saraiva Leite - 2023
- Projeto Análise de sentimentos sobre notícias do tema ESG

In [5]:
import warnings
import pandas as pd
import datetime as dt
from noticias_google_buscador import busca_noticias_google_news
from noticias_google_buscador_esg import *
import numpy as np
from noticias_io import *

warnings.filterwarnings('ignore')


In [ ]:
df_atual = le_base_noticias_bruta_para_df()
df_empresas = le_lista_empresas_para_df()

hash_noticias_ja_existentes = list(df_atual['hash'])

# processa empresas
for i, row in df_empresas[df_empresas['buscar'].str.upper() == 'SIM'].iterrows():
    
    empresa = row['empresa']
    
    aplicar_filtros= ( (not pd.isnull(row['aplicar_filtros_relevancia'])) and   row['aplicar_filtros_relevancia'].upper() == 'SIM')
    inclui_apenas_nome_empresa = ( (not pd.isnull(row['tipo_busca_exaustiva'])) and    row['tipo_busca_exaustiva'].upper() == 'SIM')
    buscar_apelidos= ( (not pd.isnull(row['buscar_apelidos'])) and   row['buscar_apelidos'].upper() == 'SIM')
    
    
    atualiza = (  (not pd.isnull(row['tipo_busca_atualizacao'])) and    row['tipo_busca_atualizacao'].upper() == 'SIM' and (not pd.isnull(row['ultima_data_publicacao'])))
    ultima_data = row['ultima_data_publicacao'].date()
    
    print('\n' +  str(dt.datetime.now())  + ' Buscando empresa ' + empresa)
    dfEmpresasListadas = df_empresas
    
    df1 = busca_noticias_google_esg(empresa, atualiza=atualiza, ultima_data = ultima_data, inclui_apenas_nome_empresa = inclui_apenas_nome_empresa)
    # verifica se faz a busca por apelidos tambem (ex banco do brasil > BB)
    if buscar_apelidos and (not pd.isnull(row['apelidos'])):
        for apelido in row['apelidos'].split(','):
            print('\n' +  str(dt.datetime.now())  + ' Buscando apelido ' + apelido + " da empresa " + empresa)
            df1_apelido = busca_noticias_google_esg(apelido, atualiza=atualiza, ultima_data = ultima_data, inclui_apenas_nome_empresa = inclui_apenas_nome_empresa)

            if df1_apelido is not None and len(df1_apelido) >0:
                if df1 is not None and len(df1) > 0:
                    df1 = pd.concat([df1,df1_apelido]).drop_duplicates(ignore_index=True)
                else:
                    df1 = df1_apelido
        
    if len(df1) >0:
        # ajustes
        #df1['data_publicacao'] = df1['data_publicacao'].dt.date
        df1['empresa'] = empresa # mantem padrao do nome caso tenham apelidos
        df1 = ajusta_nomes_empresas_dataframe(df1)

        # cria hash
        df1['hash'] = df1.apply(lambda row: criar_hash_noticia(texto=row['titulo'], empresa=row['empresa'], titulo=row['titulo'], data=row['data_publicacao']), axis=1)
        print(str(dt.datetime.now())  + ' ... noticias encontradas: ' + str(len(df1)))
    
    if len(df1) >0:
        df1 = df1[~df1['hash'].isin(hash_noticias_ja_existentes)]
        print(str(dt.datetime.now())  + ' ... noticias novas: ' + str(len(df1)))
    
    if len(df1) > 0:
        df2 = recupera_noticias_completas(df1)
        #print(df1)
        print(str(dt.datetime.now())  + ' ... textos buscados: ' + str(len(df2)))
            
        if aplicar_filtros:
            df3 = filtra_noticias_nao_relacionadas(df2, empresa)
            if len(df3) >0:
                df4 = filtra_citacao_relevante(df3, empresa, dfEmpresasListadas )
                print(str(dt.datetime.now())  + ' ... novas noticias após filtro citaçoes: ' + str(len(df4)))
                
        else:
            df4 = df2
        
        if len(df4) >0:
            df5 = classifica_textos_coletados(df4)
            df6 = df5
            print(str(dt.datetime.now())  + ' ... novas noticias após filtros: ' + str(len(df6)))
            df_atual = le_base_noticias_bruta_para_df()
            df = pd.concat([df_atual,df6]).drop_duplicates(ignore_index=True)
            df = df.drop_duplicates(['titulo','empresa','fonte', 'texto_completo'], keep='first')
            salva_base_noticias_bruta(df)


# complementando os dados
# atualiza nome, cnpj, setor
df_noticias = le_base_noticias_bruta_para_df()
df_empresas = le_lista_empresas_para_df()
df_noticias['Nome'] = df_noticias['empresa'].map(df_empresas.set_index('empresa')['Nome'])
df_noticias['CNPJ'] = df_noticias['empresa'].map(df_empresas.set_index('empresa')['CNPJ'])
df_noticias['Razão social'] = df_noticias['empresa'].map(df_empresas.set_index('empresa')['RAZÃO SOCIAL'])
df_noticias['Código'] = df_noticias['empresa'].map(df_empresas.set_index('empresa')['Código'])
df_noticias['Setor'] = df_noticias['empresa'].map(df_empresas.set_index('empresa')['SETOR'])
salva_base_noticias_bruta(df_noticias)

# atualiza ultima data de captura
df_empresas['ultima_data_publicacao'] = df_empresas['empresa'].map(df_noticias[['empresa', 'data_publicacao']].groupby('empresa').max()['data_publicacao'] )
salva_lista_empresas(df_empresas)

print('\nCaptura concluída')


2024-03-10 22:36:48.627915 Buscando empresa irb brasil re
...... buscando no google irb brasil re

2024-03-10 22:36:56.324426 Buscando apelido irb resseguros da empresa irb brasil re
...... buscando no google irb resseguros

2024-03-10 22:37:28.717891 Buscando apelido IRBR3 da empresa irb brasil re
...... buscando no google IRBR3
2024-03-10 22:38:01.002439 ... noticias encontradas: 521
2024-03-10 22:38:01.013828 ... noticias novas: 444
